In [1]:
# https://plotly.com/python/mapbox-layers/
# https://plotly.com/python/discrete-color/

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import geopandas
import plotly.express as px
from shapely.geometry import Polygon, Point
from sodapy import Socrata
from d2i_tools import *
pd.options.mode.chained_assignment = None  # default='warn'

# read live status
def read_current_parkingSensors():
    cols = ["st_marker_id", "status", 
#             "disabilityext1","duration1","effectiveonph1",
#             "endtime1","exemption1","fromday1","starttime1","today1","typedesc1",
#             "description1", "description2","description3","description4","description5","description6",
            "bay_id"]

    apptoken = os.environ.get("SODAPY_APPTOKEN")
    domain = "data.melbourne.vic.gov.au"
    client = Socrata(domain, apptoken)

    r1 = client.get_all('vh2v-4nfs')  # read current parking sensors snapshot available
    r2 = client.get_all("ntht-5rk7")  # read parking restrictions records of all bays

    df1 = pd.DataFrame.from_dict(r1)
    df2 = pd.DataFrame.from_dict(r2)
    df = pd.merge(df1, df2, left_on='bay_id', right_on='bayid', how='left')
    df = df[cols].rename(columns={'st_marker_id':'marker_id'})
    return df

def create_fullUpdate(df):
    gdf = geopandas.read_file("datasets/onStreetParkingBays_baselist_bayCentroid.geojson")
    gdf["marker_id"] = gdf["marker_id"].str.upper()
    df["marker_id"] = df["marker_id"].str.upper()
    ngdf = gdf.merge(df, how="outer", on="marker_id").drop_duplicates()
    ngdf["status"] = ngdf["status"].fillna("unknown")
    ngdf = ngdf.drop(columns=["rd_seg_id","rd_seg_dsc"])
    print(f"{ngdf.shape[0]} parking sensors in full base list.")
    return ngdf

def plot_map(df,map_height=400,map_zoom=12, marker_max=9):
    fig = px.scatter_mapbox(df, lat="lati", lon="long", hover_name="marker_id", #hover_data=["description1","description2","description3","description4","description5","description6"],
                                color="status", size="area_m2", size_max=marker_max, zoom=map_zoom, height=map_height,
                                color_discrete_map={
                                    "unknown":"gray", "Unoccupied":"green", "Present":"red"},
                                category_orders={"status": ["unknown", "Unoccupied", "Present"]})
    # fig.update_layout(mapbox_style="open-street-map")
    fig.update_layout(mapbox_style="carto-positron")
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()

In [2]:
dfrc = read_current_parkingSensors()
ngdf = create_fullUpdate(dfrc)
# plot_map(ngdf,map_height=500,map_zoom=12, marker_max=9)
getCurrentPSstatus(dfrc)

5895 parking sensors in full base list.
Number of current status records of parking sensors read : 1238
Number of current available parking bays : 969 (78.3% of total)
Number of current unavailable parking bays : 269 (21.7% of total)


(1238, 269, 969)

In [4]:
# Russell St (btw Lonsdale St & Bourke St)
lat1, lon1 = -37.8109157014938, 144.96707250378861
lat2, lon2 = -37.81086175932815, 144.9673721678719
lat3, lon3 = -37.81274371825473, 144.96821426187805
lat4, lon4 = -37.81280365246422, 144.96796390960594
lat5, lon5 = -37.81184769602218, 144.96752389652164
polygon = Polygon([(lon1, lat1), (lon2, lat2), (lon3, lat3), (lon4, lat4), (lon5, lat5)])

gdf_filtered = geoPolyClip(ngdf, polygon)
plot_map(gdf_filtered,map_height=300,map_zoom=15, marker_max=10)

dfrh = pd.read_csv("datasets/003_vh2v-4nfs__bufferred.csv", parse_dates=["db_read_time"])
dfrh_filtered = dfrh[dfrh["st_marker_id"].isin(gdf_filtered["marker_id"])]
startdt, enddt = "2021-09-09 11:00:00", "2021-09-16 11:00:00"
dfts1 = timeIntStats(dfrh_filtered, startdt, enddt, bin="15min", bin_stat="median")
dfts1a = dfts1.reset_index(level=0)
# px.scatter(dfts1a, x="time_interval", y=["Unoccupied","Present"], hover_name="time_interval")
px.bar(dfts1a, x="time_interval", y=["Unoccupied","Present"], hover_name="time_interval", 
       height=300, title="Number of parking bays",
       color_discrete_map={"unknown":"gray", "Unoccupied":"green", "Present":"red"},
       category_orders={"status": ["unknown", "Unoccupied", "Present"]})

34 parking sensors filtered.


540 time intervals in resampled data
